# Electoral district

In [1]:
import json
import pandas as pd

In [2]:
electoral_district_df = pd.read_csv('66_Vote62_election-zones.csv')

In [3]:
electoral_district_df[electoral_district_df.duplicated(['จังหวัด', 'อำเภอ', 'ตำบล'])]

,จังหวัด,อำเภอ,ตำบล,เขตเลือกตั้ง,index,เทศบาล,เขตเลือกตั้ง.1
1992,นครพนม,เมืองนครพนม,หนองแสง,2,นครพนมเมืองนครพนมหนองแสง,เทศบาลเมืองนครพนม,2
1993,นครพนม,เมืองนครพนม,อาจสามารถ,2,นครพนมเมืองนครพนมอาจสามารถ,เทศบาลเมืองนครพนม,2
2364,นครศรีธรรมราช,ท่าศาลา,ท่าศาลา,9,นครศรีธรรมราชท่าศาลาท่าศาลา,เทศบาลตำบลท่าศาลา,9
2435,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ท่าไร่,1,นครศรีธรรมราชเมืองนครศรีธรรมราชท่าไร่,เทศบาลตำบลปากนคร,1
2441,นครศรีธรรมราช,เมืองนครศรีธรรมราช,บางจาก,1,นครศรีธรรมราชเมืองนครศรีธรรมราชบางจาก,เทศบาลตำบลบางจาก,1
2443,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ปากนคร,1,นครศรีธรรมราชเมืองนครศรีธรรมราชปากนคร,เทศบาลตำบลปากนคร,1
2445,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ปากพูน,2,นครศรีธรรมราชเมืองนครศรีธรรมราชปากพูน,เทศบาลตำบลท่าแพ,2
2448,นครศรีธรรมราช,เมืองนครศรีธรรมราช,นาเคียน,2,นครศรีธรรมราชเมืองนครศรีธรรมราชนาเคียน,NaN,2
2449,นครศรีธรรมราช,เมืองนครศรีธรรมราช,โพธิ์เสด็จ,1,นครศรีธรรมราชเมืองนครศรีธรรมราชโพธิ์เสด็จ,เทศบาลนครนครศรีธรรมราช,1
2450,นครศรีธรรมราช,เมืองนครศรีธรรมราช,นาเคียน,1,นครศรีธรรมราชเมืองนครศรีธรรมราชนาเคียน,เทศบาลนครนครศรีธรรมราช,1


In [4]:
electoral_district_df = electoral_district_df.assign(
    pk=electoral_district_df.apply(lambda row: "{}-{}".format(row['จังหวัด'], row['เขตเลือกตั้ง.1']), axis=1))

In [5]:
district_list = list()
for (province,district,subdistrict,thesaban), gdf in (electoral_district_df
                                                      .fillna('')
                                                      .groupby(['จังหวัด', 'อำเภอ', 'ตำบล', 'เทศบาล'])):
    district_list.append(dict(
        province=province,
        district=district,
        subDistrict=subdistrict,
        thesaban=thesaban,
        electoralFk=gdf['pk'].tolist(),
    ))

In [6]:
belongs_to_one_electoral_district = (electoral_district_df.groupby(['จังหวัด', 'อำเภอ',])['pk'].apply(
    lambda elect_dist: len(elect_dist.unique()) == 1
))

In [7]:
belongs_to_one_electoral_district.loc[('กระบี่', 'อ่าวลึก')]

True

In [8]:
electdistrict = dict()
for (province, elec_num, elecPk), gdf in electoral_district_df.groupby(['จังหวัด', 'เขตเลือกตั้ง.1', 'pk']):
    _district_list=[]
    for (district), subdist_df in gdf.groupby(['อำเภอ']):
        sub_districts = []
        for subd, thesaban in subdist_df[['ตำบล','เทศบาล']].values:
            if pd.isna(thesaban):
                sub_districts.append(subd)
            else:
                sub_districts.append(f'{subd} ({thesaban})')

        _district_list.append(dict(
            name=district,
            subDistricts=sub_districts,
            belongsToOneElecDist=bool(belongs_to_one_electoral_district.loc[(province,district)]))
        )
    electdistrict[elecPk] = dict(
        province=gdf['จังหวัด'].tolist()[0],
        districts=_district_list,
        electoralDistrictNumber=int(elec_num),
    )

In [9]:
with open('district_province_list.json', 'w') as fp:
    json.dump(district_list, fp, ensure_ascii=False)

In [10]:
with open('electoral_district_table.json', 'w') as fp:
    json.dump(electdistrict, fp, ensure_ascii=False)

# Candidates data

In [11]:
import re
import requests
import os
import numpy as np

In [12]:
electoral_dir = 'electorals'
os.makedirs(electoral_dir, exist_ok=True)

In [13]:
candidates_df = pd.read_csv('66_WV_Candidates.csv')
party_number_df = pd.read_csv('66_WV_Party.csv')
pm_candidates_df = pd.read_csv('66_WV_PMCandidate.csv')

In [14]:
candidates_df.loc[candidates_df.zone.isna(), 'zone'] = -1
# change data type
candidates_df.loc[:,'zone'] = candidates_df.zone.astype(int)

In [15]:
# set party with no number to -1
party_number_df.loc[:, 'partylist_no'] = party_number_df.partylist_no.apply(
    lambda x: int(x) if str(x).isdigit() else -1)

In [16]:
parties_table_ep = (
    'https://sheets.wevis.info/api/v1/db/public/shared-view/'
    '40065196-c978-4d7a-b3fb-fb84694383a7'
    '/rows'
    '?'
    'l=900'
    '&f=Id,Name,PartyType,PartyGroup,Images,IsActive,Website'
)
resp = requests.get(parties_table_ep)
parties_table = resp.json()['data']['list']

In [17]:
people_party_history_ep = (
    'https://sheets.wevis.info/api/v1/db/public/shared-view/'
    '572c5e5c-a3d8-440f-9a70-3c4c773543ec'
    '/rows'
    '?'
    'nested[PeoplePartyHistory][fields]=Party,EstablishedDate'
    '&'
    'f=Name,PeoplePartyHistory,Id,Images'
    '&'
    'l=900')
resp = requests.get(people_party_history_ep)
people_history_table = resp.json()['data']['list']

In [18]:
def get_image_url(obj):
    if obj['Images']:
        image = obj['Images'][0]
        if 'url' in image.keys():
            return image['url']
        if 'path' in image.keys():
            return 'https://sheets.wevis.info/'+image['path']

In [19]:
people_history_table

[{'Name': 'ประยุทธ์ จันทร์โอชา',
  'Id': 1,
  'Images': [{'title': 'ประยุทธ์-จันทร์โอชา.jpg',
    'url': 'https://sheets.wevis.info/download/noco/They-Work-For-Us/People/Images/ประยุทธ์-จันทร์โอชา.jpg',
    'mimetype': 'image/jpg'}],
  'PeoplePartyHistory': [{'EstablishedDate': '2023-02-27',
    'Party': {'Id': 37, 'Name': 'รวมไทยสร้างชาติ'}}]},
 {'Name': 'ประวิตร วงษ์สุวรรณ',
  'Id': 2,
  'Images': [{'title': 'ประวิตร-วงษ์สุวรรณ.jpg',
    'url': 'https://sheets.wevis.info/download/noco/They-Work-For-Us/People/Images/ประวิตร-วงษ์สุวรรณ.jpg',
    'mimetype': 'image/jpg'}],
  'PeoplePartyHistory': [{'EstablishedDate': '2022-12-01',
    'Party': {'Id': 20, 'Name': 'พลังประชารัฐ'}}]},
 {'Name': 'สมคิด จาตุศรีพิทักษ์',
  'Id': 3,
  'Images': [{'title': 'สมคิด-จาตุศรีพิทักษ์.jpg',
    'url': 'https://sheets.wevis.info/download/noco/They-Work-For-Us/People/Images/สมคิด-จาตุศรีพิทักษ์.jpg',
    'mimetype': 'image/jpg'}],
  'PeoplePartyHistory': [{'EstablishedDate': '2019-07-10',
    'Party': {

In [20]:
people_image_url = {p['Name']:get_image_url(p) for p in people_history_table}

In [21]:
parties_table

[{'Name': 'สภาผู้แทนราษฎร',
  'PartyType': 'สส',
  'PartyGroup': None,
  'Website': 'https://www.parliament.go.th',
  'IsActive': True,
  'Id': 1,
  'Images': None},
 {'Name': 'วุฒิสภา',
  'PartyType': 'สว',
  'PartyGroup': None,
  'Website': 'https://www.senate.go.th',
  'IsActive': True,
  'Id': 2,
  'Images': [{'path': 'download/noco/They-Work-For-Us/Parties/Images/48JaSVe38lyzq_lsBN.jpg',
    'title': 'senate.jpg',
    'mimetype': 'image/jpeg',
    'size': 2564}]},
 {'Name': 'คณะรัฐมนตรี',
  'PartyType': 'ครม',
  'PartyGroup': None,
  'Website': 'https://www.thaigov.go.th',
  'IsActive': True,
  'Id': 3,
  'Images': None},
 {'Name': 'ครูไทยเพื่อประชาชน',
  'PartyType': 'พรรค',
  'PartyGroup': 'ร่วมรัฐบาล',
  'Website': None,
  'IsActive': True,
  'Id': 4,
  'Images': [{'path': 'download/noco/They-Work-For-Us/Parties/Images/EYeQ37B_15pvPoy8Nb.png',
    'title': 'ครูไทยเพื่อประชาชน.png',
    'mimetype': 'image/png',
    'size': 25820}]},
 {'Name': 'ชาติไทยพัฒนา',
  'PartyType': 'พรรค

In [22]:
# create table
def create_party_data():
    party_data = dict()
    
    # helper
    def get_party_data(data_getter):
        data = dict()
        for party in parties_table:
            temp = data_getter(party)
            # filter None value
            if temp is None: continue
            data[party['Name']] = data_getter(party)
        return data
    
    def get_party_image_url():
        data = dict()
        def getter(party):
            return get_image_url(party)
        return get_party_data(getter)
    
    party_data['image_url'] = get_party_image_url()
    
    def get_party_number():
        return party_number_df.set_index('party').to_dict()['partylist_no']
    party_data['number'] = get_party_number()
    
    
    def get_party_website():
        def getter(party):
            return party['Website']
        return get_party_data(getter)
    party_data['website'] = get_party_website()
    
    def get_party_candidates():
        data = dict()
        for party, party_df in pm_candidates_df.sort_values('order').groupby('party'):
            data[party] = [{'Name': name, 'Image': people_image_url.get(name)} for name in party_df.name]
        return data
    party_data['pm_candidates'] = get_party_candidates()
    
    def get_party_promisetracker_url():
        data = dict()
        pt_parties_url = 'https://raw.githubusercontent.com/wevisdemo/promise-tracker/main/data/parties.json'
        resp = requests.get(pt_parties_url)
        promisetracker_parties = resp.json()
        pt_url = 'https://promisetracker.wevis.info/explore/?party={}'
        for party in promisetracker_parties:
            party_name_mabe_many = party['name']
            for party_name in party_name_mabe_many.split('/'): # อนาคตใหม่/ก้าวไกล
                data[party_name] = pt_url.format(party_name_mabe_many)
        return data
    party_data['promisetracker_urls'] = get_party_promisetracker_url()
    
    def get_party_law_watch_url():
        resp = requests.get('https://raw.githubusercontent.com/wevisdemo/law-watch/main/src/data/parties.ts')
        law_watch_parties_string = re.findall('\[[^\]]*\]', # find arrays
                                              resp.content.decode().replace('\n',' ')) # remove new lines
        data = dict()
        for list_string in law_watch_parties_string[:2]:
            party_list = json.loads(re.sub('\'', '"',list_string))
            for party in party_list:
                data[party] = 'https://wevis.info/law-watch#investigate-section'
        return data
    party_data['law_watch_urls'] = get_party_law_watch_url()
    
    def get_party_theyworkforus_url():
        def getter(party):
            # if it was government or oppositon
            if party['PartyGroup']:
                party_name = party['Name']
                return f'https://theyworkforus.wevis.info/party/{party_name}'
        return get_party_data(getter)
    party_data['theyworkforus_urls'] = get_party_theyworkforus_url()
    
    def get_party_is_government():
        def getter(party):
            if party['PartyGroup'] == 'ร่วมรัฐบาล':
                return True
            if party['PartyGroup'] == 'ฝ่ายค้าน':
                return False
        return get_party_data(getter)
    party_data['is_government'] = get_party_is_government()
    
    def get_party_is_opposition():
        def getter(party):
            if party['PartyGroup'] == 'ร่วมรัฐบาล':
                return False
            if party['PartyGroup'] == 'ฝ่ายค้าน':
                return True
        return get_party_data(getter)
    party_data['is_opposition'] = get_party_is_opposition()
                
    return party_data

In [23]:
party_data = create_party_data()

In [24]:
parties_table

[{'Name': 'สภาผู้แทนราษฎร',
  'PartyType': 'สส',
  'PartyGroup': None,
  'Website': 'https://www.parliament.go.th',
  'IsActive': True,
  'Id': 1,
  'Images': None},
 {'Name': 'วุฒิสภา',
  'PartyType': 'สว',
  'PartyGroup': None,
  'Website': 'https://www.senate.go.th',
  'IsActive': True,
  'Id': 2,
  'Images': [{'path': 'download/noco/They-Work-For-Us/Parties/Images/48JaSVe38lyzq_lsBN.jpg',
    'title': 'senate.jpg',
    'mimetype': 'image/jpeg',
    'size': 2564}]},
 {'Name': 'คณะรัฐมนตรี',
  'PartyType': 'ครม',
  'PartyGroup': None,
  'Website': 'https://www.thaigov.go.th',
  'IsActive': True,
  'Id': 3,
  'Images': None},
 {'Name': 'ครูไทยเพื่อประชาชน',
  'PartyType': 'พรรค',
  'PartyGroup': 'ร่วมรัฐบาล',
  'Website': None,
  'IsActive': True,
  'Id': 4,
  'Images': [{'path': 'download/noco/They-Work-For-Us/Parties/Images/EYeQ37B_15pvPoy8Nb.png',
    'title': 'ครูไทยเพื่อประชาชน.png',
    'mimetype': 'image/png',
    'size': 25820}]},
 {'Name': 'ชาติไทยพัฒนา',
  'PartyType': 'พรรค

In [25]:
DISSOVED_DATE = '2023-03-01'
person_is_government = dict()
person_is_opposition = dict()
for person in people_history_table:
    for party in person['PeoplePartyHistory']:
        if party['EstablishedDate'] is not None and (party['EstablishedDate'] > DISSOVED_DATE):
            person['PeoplePartyHistory'].remove(party)
        else:
            party_id = party['Party']['Id']
            party_name = party['Party']['Name']
            party_group = party_data['is_government'].get(party_name)
            person_is_government[person['Name']] =\
                person_is_government.get(person['Name'], False) or party_data['is_government'].get(party_name)
            person_is_opposition[person['Name']] =\
                person_is_opposition.get(person['Name'], False) or party_data['is_opposition'].get(party_name)

In [26]:
def get_candidate_iamge_path():
    with open('./candidates/candidate_data.json') as f:
        data = json.load(f)
    return {name: info['image'] for name, info in data.items()}

people_image_url.update(get_candidate_iamge_path())

In [27]:
def people_get_image(name):
    return people_image_url.get(name)

In [28]:
def get_smartvote_candidate_info():
    with open('./smart-vote/smartvote-candidates.json') as fp:
        candidates = json.load(fp)
    data = dict()
    not_votable = dict()
    for candidate in candidates:
        if candidate['PartyName'] == '0':
            candidate['PartyName'] = 'มิติใหม่'
        candidate['PartyName'] = candidate['PartyName'].strip()
        name = candidate['ProvinceName']+str(candidate['ZoneNumber'])+candidate['PartyName']
        data[name] = candidate
    return data
people_info = get_smartvote_candidate_info()

In [29]:
skip_people = [
    'เขมจิรา พชรจรวยพร',
    'สมศักดิ์ สาระชาติ',
    'สุลาวัลย์ รักชลธี',
    'มูฮัมหมัด นอร์แมน กานตี้',
    'ชยรพ อ่องยิ้ม',
    'สุธินี ธัมวิสุทธิวรากร',
    'นิดา ขนายงาม',
    'วิมาน หนูท้าว',
    'สมคิด มงคลส่ง', # ใน smart vote เบอร์ 7 คือ กัมปนาท ธิสา ประชาธิปัตย์
    'ภาณุพล จอมศรี',
    'พลวรรฒน์ ธรรมโสภณ',
    'พิเชษฐ รัตนพันธ์',
    'อับดุลเราะมัน มอลอ',
    'ธนากรณ์ บุญมิ่ง',
    'มีชัย เทศนอก',
    'รัชฏะ สมรทินกร',
    'อดุลย์ ณ ลำปาง',
    'ถนอม ผุยผง',
    'โสภณ ก้องเสนาะ',
    'สมพงค์ กิจประพันธ์',
]

In [30]:
people_info['กรุงเทพมหานคร10คลองไทย']

{'CandidateId': 'weffewfedudhx289fhm8few',
 'Title': 'ถอนชื่อ',
 'FirstName': 'ตาม รธน. 2560 ม.87',
 'LastName': 'วรรคสอง',
 'PartyName': 'คลองไทย',
 'PartyNumber': '189',
 'CandidateNo': '16',
 'Votable': True,
 'ProvinceName': 'กรุงเทพมหานคร',
 'ProvinceCode': 10,
 'ZoneNumber': 10,
 'Id': 'weffewfedudhx289fhm8few',
 'Age': '47',
 'Occupation': 'ธุรกิจส่วนตัว',
 'HighestEducation': 'ปริญญาตรี'}

In [31]:
people_list = []
party_table = dict()
people_problem = []
for i, row in candidates_df.iterrows():
    if row.party not in party_table.keys():
        party_table[row.party] = dict(
            Name=row.party,
            Number=party_data['number'][row.party],
            Image=party_data['image_url'].get(row.party),
            Candidate=party_data['pm_candidates'].get(row.party, []),
            PastGovernment=party_data['is_government'].get(row.party, False),
            PastOpposition=party_data['is_opposition'].get(row.party, False),
            # TODO: fix this on production
            Policy=f'https://election66.wevis.info/policyshop/party/{row.party}',
            Promise=party_data['promisetracker_urls'].get(row.party),
            Law=party_data['law_watch_urls'].get(row.party),
            Others=party_data['theyworkforus_urls'].get(row.party),
            Website=party_data['website'].get(row.party),
            PartyList=list()
        )
    is_past_mp = person_is_government.get(row['name'], False) or person_is_opposition.get(row['name'], False)
    if row.mptype == 'เขต':
        candidate_smartvote_key = row.province + str(row.zone) + row.party
        if (candidate_smartvote_key not in people_info.keys() and
            row['name'] not in skip_people):
            print(candidate_smartvote_key,repr(row.to_dict()) + ' not in smart vote')
            people_problem.append(row.to_dict())
        
        candidate_smartvote = people_info.get(candidate_smartvote_key, {})
        candidate_is_invalid = (candidate_smartvote.get('Title') in ['ถอนชื่อ','ผู้สมัคร','ประกาศ',])
            
        people_list.append(dict(
            Name=row['name'],
            Number=row['number'],
            Birthdate=None,#data_getter('Birthdate', row['name']), # TODO:
            Age=candidate_smartvote.get('Age'),
            Education=candidate_smartvote.get('HighestEducation'),
            ExOccupation=candidate_smartvote.get('Occupation'),
            Party=row.party,
            Province=row.province,
            Zone=row.zone,
            Image=people_get_image(row['name']),
            PastMP=is_past_mp,
            PastGovernment=person_is_government.get(row['name'], False),
            PastOpposition=person_is_opposition.get(row['name'], False),
            Invalid=candidate_is_invalid
        ))
    elif row.mptype == 'บัญชีรายชื่อ':
        party_table[row.party]['PartyList'].append(dict(
            Name=row['name'],
            Number=row.number,
            PastMP=is_past_mp,
        ))

แม่ฮ่องสอน1ประชาไทย {'number': 10, 'title_name': 'นายทองสุข ติโต๊ะ', 'title': 'นาย', 'name': 'ทองสุข ติโต๊ะ', 'party': 'ประชาไทย', 'mptype': 'เขต', 'province': 'แม่ฮ่องสอน', 'zone': 1, 'gender': 'male', 'birthdate*': nan, 'education*': nan, 'exoccupation*': nan} not in smart vote


In [32]:
for dist_key, district in electdistrict.items():
    electoral_people_list = [p for p in people_list
                             if p['Province'] == district['province'] # filter out
                             and p['Zone'] == district['electoralDistrictNumber']]
    
#     electoral_parties = [p['Party'] for p in electoral_people_list if p['Number'] != -1]
#     electoral_parties.sort(key=lambda p: party_table[p]['Number'])
    file_path = os.path.join(electoral_dir,f'{dist_key}.json')
    electoral_data = dict(People=sorted(electoral_people_list,key=lambda p: p['Number']),)
    
    with open(file_path,'w') as fp:
        json.dump(electoral_data, fp, ensure_ascii=False)

In [33]:
with open('parties.json', 'w') as fp:
    json.dump(party_table, fp, ensure_ascii=False)